*Created by Petteri Nevavuori <<petteri.nevavuori@gmail.com>>.*

---

# Building a General Purpose Blockchain

In this notebook we'll build a general purpose blockhain. We'll showcase the fundamental aspects of block-related functionalities. The implementation follows closely on the premises of course Blockchain A-Z found in Udemy but not identically. This is the first module of the course. We will utilize the Python-based Flask web-server package to serve the general purpose blockchain. The development itself will be separated to several stages.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Building-the-Blockhain" data-toc-modified-id="1.-Building-the-Blockhain-1">1. Building the Blockhain</a></span></li><li><span><a href="#2.-Mining-the-Blockchain" data-toc-modified-id="2.-Mining-the-Blockchain-2">2. Mining the Blockchain</a></span><ul class="toc-item"><li><span><a href="#2.1-Creating-the-Web-App" data-toc-modified-id="2.1-Creating-the-Web-App-2.1">2.1 Creating the Web App</a></span></li><li><span><a href="#2.2-Testing-the-Blockchain-App" data-toc-modified-id="2.2-Testing-the-Blockchain-App-2.2">2.2 Testing the Blockchain App</a></span></li></ul></li></ul></div>

## 1. Building the Blockhain

The first thing of course is to build the blockchain. This blockchain utilizes a [proof-of-work](https://en.wikipedia.org/wiki/Proof-of-work_system) (POW) to validate the blocks. Essentially the POW states a problem that requires work to solve it and a target level for acceptance of the proof. With Bitcoins this means iteratively brute-forcing through [nonces](https://en.wikipedia.org/wiki/Cryptographic_nonce) to find the [SHA-256 hash](https://en.wikipedia.org/wiki/SHA-2) with four leading zeros. The probability of selecting a proper hexadecimal string with of 64 characters randomly is:

In [9]:
print("{:.5f}%".format(int('0000'+'F'*60,16)/int('F'*64,16)*100))

0.00153%


Our implementation's `proof` of a block essentially equals the nonce. This means that to produce a valid proof a number of iterations have to be tried out.

In our blockchain the block-wise proofs are calculated with the proof values of the previous block. This means that changing the a proof value mid-chain would require re-calculating i.e. re-mining the blocks to have new proofs and thus re-calculating the block-wise hashes. Because our initial blockchain does not have any data associated with it just yet, linking the proofs and blocks separately serves as a safety measure to ensure the tampering of our distributed ledger would be nearly impossible.

In [10]:
import datetime
import hashlib
import json


class Blockchain:

    def __init__(self):
        self.chain = []
        self.create_block(proof=1, previous_hash='0')

    def create_block(self, proof, previous_hash):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': datetime.datetime.now(),
            'proof': proof,
            'previous_hash': previous_hash,
        }
        self.chain.append(block)
        return block

    def get_previous_block(self):
        return self.chain[-1]

    def hash_proof(self, previous_proof, next_proof):
        "Calculate the SHA256-hash"
        return (hashlib
                .sha256(str(new_proof**2 - previous_proof**2).encode())
                .hexdigest())

    def hash_block(self, block):
        "Calculate the SHA256-hash for a block."

        encoded_block = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encoded_block).hexdigest()

    def proof_of_work(self, previous_proof):
        "Calculate a new proof related to the previous block."

        next_proof = 1
        check_proof = False

        while check_proof is False:

            if self.hash_proof(previous_proof, next_proof)[:4] == '0000':
                check_proof = True

            else:
                next_proof += 1

        return next_proof

    def is_chain_valid(self, chain):
        "Validate that block and proof hashes are correct across the chain."

        for i in range(len(chain)):

            if i == 0:
                continue

            if chain[i]['previous_hash'] != self.hash_block(chain[i-1]):
                return False

            previous_proof = chain[i-1]['proof']
            next_proof = chain[i]['block']

            if self.hash_proof(previous_proof, next_proof)[:4] == '0000':
                return False

            return True

## 2. Mining the Blockchain

Next we implement a simple web application, through which we will then start mining blocks for our general purpose chain.

### 2.1 Creating the Web App

We will use Flask to create an accessible chain that is able to handle GET and POST requests for our chain accordingly. We will use threading to run the miniserver in a non-blocking manner. Otherwise the notebook would freeze until the server was shutdown, essentially requiring shutdown calls from outside this notebook.

With a POST request we mine a block and save it to the chain.

In [ ]:
import threading
import requests
import time

from flask import Flask, request, jsonify
import werkzeug

app = Flask(__name__)


@app.route('/blocks', methods=['GET', 'POST'])
def blocks():

    if requests.method == 'POST':

        prev_block = blockchain.get_previous_block()
        prev_hash = blockchain.hash_block(prev_block)
        prev_proof = prev_block['proof']

        proof = blockchain.proof_of_work(prev_proof)

        block = blockchain.create_block(proof, prev_hash)

        response = {
            'message': 'Congratulations, you just mined a Block!',
            'block': block
        }

        return (jsonify(response), 200)

    if requests.method == 'GET':

        return 'Not implemented yet'


@app.route('/shutdown')
def shutdown():

    request.environ.get('werkzeug.server.shutdown')()
    return "Shutting down"

We will also use a distinct class to easen up the use of the threaded server.

In [22]:
class BlockchainApp:

    def __init__(self):

        self.blockchain = Blockchain()
        self.t = threading.Thread(target=app.run, kwargs={
                                  'host': 'localhost', 'port': 5000})

    def start(self):
        print("Starting the Blockchain miniserver")
        self.t.start()

    def stop(self):
        if self.t.is_alive():
            response = requests.get('http://localhost:5000/shutdown')
            print(response)

### 2.2 Testing the Blockchain App

Let's then give the app a test run!

In [23]:
blockchain_app = BlockchainApp()
blockchain_app.start()
blockchain_app.stop()

 * Running on http://localhost:5000/ (Press CTRL+C to quit)


Starting the Blockchain miniserver


127.0.0.1 - - [10/Apr/2018 16:01:43] "GET /shutdown HTTP/1.1" 200 -


<Response [200]>


Works like a prayer.